# Generate reference solution

In [1]:
import numpy as np
import torch
from torch import nn
from typing import Tuple
import json

In [2]:
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

In [3]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_file = "Initial_NN.torch"

In [4]:
from beam import Beam, Prob_Solv_Modes, In_Cond

In [5]:
par = read_json('par.json')

Lx = par['x']
t = par['t']
n = par['n']

num_hidden = par['n_layers']
dim_hidden = par['n_neurons']

In [6]:
my_beam = Beam(Lx, 68e9, 2700, 8e-3, 40e-3, n)

In [7]:
prob = Prob_Solv_Modes(my_beam)

gamma_max = 100 # gamma_max must be increased, because spatial eigenfrequencies increase, since the beam is very short

prob.pass_g_max(gamma_max)
eig_gam = prob.find_eig()

my_beam.gamma = np.array(eig_gam)
my_beam.update_freq()

# Just one parameter independent for gamma (order of the system reduced)
F = prob.find_all_F(my_beam)
prob.update_gamma(my_beam)
phi = prob.return_modemat(F)
my_beam.update_phi(phi)

In [8]:
my_In_Cond = In_Cond(my_beam)

w_0 = my_beam.phi[:,0]
w_dot_0 = np.zeros(len(w_0))

my_In_Cond.pass_init_cond(w_0, w_dot_0)
A, B = my_In_Cond.compute_coeff()

t_lin = np.linspace(0, 1, n)
my_beam.calculate_solution(A, B, t_lin)
w = my_beam.w

In [9]:
def adimensionalize_sol(w: np.ndarray, w_ast: float):
    return w/w_ast

w_ad = adimensionalize_sol(w, Lx).T

correct to transpose it? based on how reshape works
- columns of w are referred to different time points

# Setup NN for initial conditions

## Define training dataset

In [10]:
x_lin = np.linspace(0, 1, n)

x, t = np.meshgrid(x_lin, t_lin)
x = x.reshape(-1)
t = t.reshape(-1)
X = np.stack((x, t), axis=1)

## Split data for validation

In [11]:
y = w.reshape(-1, 1)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Prepare data for pytorch

In [13]:
x_train = X_train[:, 0].reshape(-1, 1)
t_train = X_train[:, 1].reshape(-1, 1)

x_val = X_val[:, 0].reshape(-1, 1)
t_val = X_val[:, 1].reshape(-1, 1)

In [14]:
x = torch.tensor(x_train).to(device).float()
t = torch.tensor(t_train).to(device).float()

x_val = torch.tensor(x_val).to(device).float()
t_val = torch.tensor(t_val).to(device).float()

y = torch.tensor(y_train).to(device).float()
y_val = torch.tensor(y_val).to(device).float()

In [15]:
from nn import *

In [16]:
nn = NN(num_hidden, dim_hidden, dim_input = 2, dim_output = 1).to(device)

In [17]:
from tqdm import tqdm
from typing import Callable
import pytz

epochs = 8000
lr = 0.002

loss_fn = Loss(
    x,
    t,
    y
)

In [18]:
nn_trained, loss_values = train_model(
    nn, loss_fn=loss_fn, learning_rate=lr, max_epochs=epochs, x_val=x_val, t_val=t_val, y_val=y_val)

Loss: 0.1168: : 8001it [00:24, 325.33it/s]                        


In [19]:
import os
import pytz
import datetime
from write_logs import get_current_time

folder = 'in_model'
time = get_current_time(fmt='%m-%d %H:%M')
model_name = f'{get_current_time()}.pth'

model_path = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

torch.save(nn_trained.state_dict(), model_path)

# To be checked
- ~correct association of meshpoints with solution?~